In [1]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import models
import torch.nn as nn
import torch.optim as optim
from triplet_utils import *
from utils import *
from torch.utils.data import random_split
import optuna
import torch

/home/sadat/miniconda3/envs/cowbytes/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# TripletModel definition
class TripletModel(nn.Module):
    def __init__(self):
        super(TripletModel, self).__init__()
        self.base_model = models.resnet50(pretrained=True)
        self.base_model.fc = nn.Linear(self.base_model.fc.in_features, 128)  # Adjust output size for embedding

    def forward(self, x):
        # Get the embedding directly from the base model
        return self.base_model(x)  # Call the model with input x

class TripletClassifier(nn.Module):
    def __init__(self, triplet_head, output_size=1):
        super(TripletClassifier, self).__init__()
        self.triplet_head = triplet_head
        self.classifier = nn.Linear(self.triplet_head.base_model.fc.out_features, output_size)  # Binary classification (logits)

    def forward(self, x):
        embedding = self.triplet_head(x)  # Output of base model
        class_output = self.classifier(embedding)  # Output for classification
        return embedding, class_output

In [3]:
def custom_collate(batch):
    # Unpack the batch
    anchors, positives, negatives, labels = zip(*batch)

    # Stack tensors to create batches
    anchors = torch.stack(anchors)
    positives = torch.stack(positives)
    negatives = torch.stack(negatives)

    # Convert labels to a tensor
    labels = torch.tensor(labels, dtype=torch.float32).unsqueeze(1)

    return anchors, positives, negatives, labels

In [4]:
# Load data
data_dir = '../../data/BiteCount/salient_poses/'

# Define transformations for training set and validation set
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resizing to square input
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resizing to square input
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load dataset
full_dataset = TripletDataset(data_dir, transform=train_transforms)

# Define the split ratio
train_size = int(0.8 * len(full_dataset))  # 80% for training
val_size = len(full_dataset) - train_size  # 20% for validation

# Split the dataset into training and validation sets
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Create DataLoader for train and validation
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=custom_collate)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, collate_fn=custom_collate)

def objective(trial):
    # Suggest hyperparameters
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    momentum = trial.suggest_uniform('momentum', 0.5, 0.99)
    triplet_margin = trial.suggest_uniform('triplet_margin', 0.1, 2.0)
    classification_loss_weight = trial.suggest_uniform('classification_loss_weight', 0.1, 1.0)

    print(f"\nTrial {trial.number}:")
    print(f"Suggested Hyperparameters: learning_rate={learning_rate}, momentum={momentum}, "
          f"triplet_margin={triplet_margin}, classification_loss_weight={classification_loss_weight}")
    
    # Initialize model
    triplet_model = TripletModel()
    model = TripletClassifier(triplet_model)
    model = model.to(device)

    # Define loss functions and optimizer
    criterion_classification = nn.BCEWithLogitsLoss()
    criterion_triplet = nn.TripletMarginLoss(margin=triplet_margin)
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

    # Training loop
    num_epochs = 25
    best_epoch_val_loss = float('inf')

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for batch in train_loader:
            # Unpack the batch correctly
            anchor, positive, negative, labels = batch
            
            # Move the tensors to the appropriate device
            anchor = anchor.to(device)
            positive = positive.to(device)
            negative = negative.to(device)
            labels = labels.to(device)  # Ensure labels are tensors

            # Forward pass
            anchor_embedding, anchor_output = model(anchor)
            positive_embedding, _ = model(positive)
            negative_embedding, _ = model(negative)

            # Calculate losses
            classification_loss = criterion_classification(anchor_output, labels)
            triplet_loss = criterion_triplet(anchor_embedding, positive_embedding, negative_embedding)

            # Combine losses with weighting
            loss = classification_loss_weight * classification_loss + triplet_loss
            running_loss += loss.item()

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Validation step
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch in val_loader:
                anchor, positive, negative, labels = batch
                anchor = anchor.to(device)
                positive = positive.to(device)
                negative = negative.to(device)
                labels = labels.to(device)

                anchor_embedding, anchor_output = model(anchor)
                positive_embedding, _ = model(positive)
                negative_embedding, _ = model(negative)
                classification_loss = criterion_classification(anchor_output, labels)
                triplet_loss = criterion_triplet(anchor_embedding, positive_embedding, negative_embedding)

                val_loss += (classification_loss).item()

        avg_val_loss = val_loss / len(val_loader)

        # Check for improvement
        if avg_val_loss < best_epoch_val_loss:
            best_epoch_val_loss = avg_val_loss

        print(f"Epoch {epoch + 1}/{num_epochs}: "
              f"Validation Loss: {avg_val_loss:.4f}")

    return best_epoch_val_loss  # Return the average validation loss

# Create a study object and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)  # Adjust the number of trials as needed

# Print the best hyperparameters and validation loss
print("Best hyperparameters: ", study.best_params)
print("Best validation loss: ", study.best_value)

[I 2024-10-03 19:39:18,676] A new study created in memory with name: no-name-6e1db1cf-099c-484a-aaf1-1d56421fe055
/tmp/ipykernel_25998/3782639035.py:34: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
/tmp/ipykernel_25998/3782639035.py:35: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  momentum = trial.suggest_uniform('momentum', 0.5, 0.99)
/tmp/ipykernel_25998/3782639035.py:36: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  triplet_margin = trial.suggest_uniform('triplet_margin', 0


Trial 0:
Suggested Hyperparameters: learning_rate=0.00010513056948022352, momentum=0.7687324015832673, triplet_margin=0.5734419515063479, classification_loss_weight=0.42324119802267746
Epoch 1/25: Validation Loss: 0.7032
Epoch 2/25: Validation Loss: 0.7043
Epoch 3/25: Validation Loss: 0.7068
Epoch 4/25: Validation Loss: 0.6995
Epoch 5/25: Validation Loss: 0.7013
Epoch 6/25: Validation Loss: 0.7032
Epoch 7/25: Validation Loss: 0.7052
Epoch 8/25: Validation Loss: 0.7063
Epoch 9/25: Validation Loss: 0.7010
Epoch 10/25: Validation Loss: 0.7003
Epoch 11/25: Validation Loss: 0.7007
Epoch 12/25: Validation Loss: 0.7062
Epoch 13/25: Validation Loss: 0.7006
Epoch 14/25: Validation Loss: 0.6985
Epoch 15/25: Validation Loss: 0.7026
Epoch 16/25: Validation Loss: 0.6973
Epoch 17/25: Validation Loss: 0.6951
Epoch 18/25: Validation Loss: 0.7027
Epoch 19/25: Validation Loss: 0.6990
Epoch 20/25: Validation Loss: 0.6964
Epoch 21/25: Validation Loss: 0.7030
Epoch 22/25: Validation Loss: 0.6949
Epoch 23/

[I 2024-10-03 19:41:27,406] Trial 0 finished with value: 0.6948638677597045 and parameters: {'learning_rate': 0.00010513056948022352, 'momentum': 0.7687324015832673, 'triplet_margin': 0.5734419515063479, 'classification_loss_weight': 0.42324119802267746}. Best is trial 0 with value: 0.6948638677597045.


Epoch 25/25: Validation Loss: 0.6971

Trial 1:
Suggested Hyperparameters: learning_rate=0.0005600848988312334, momentum=0.6327792322084209, triplet_margin=0.6189066125747088, classification_loss_weight=0.5434323177802199
Epoch 1/25: Validation Loss: 0.6977
Epoch 2/25: Validation Loss: 0.6972
Epoch 3/25: Validation Loss: 0.6996
Epoch 4/25: Validation Loss: 0.7032
Epoch 5/25: Validation Loss: 0.6966
Epoch 6/25: Validation Loss: 0.6984
Epoch 7/25: Validation Loss: 0.6915
Epoch 8/25: Validation Loss: 0.6982
Epoch 9/25: Validation Loss: 0.6996
Epoch 10/25: Validation Loss: 0.6983
Epoch 11/25: Validation Loss: 0.6935
Epoch 12/25: Validation Loss: 0.6974
Epoch 13/25: Validation Loss: 0.6998
Epoch 14/25: Validation Loss: 0.6965
Epoch 15/25: Validation Loss: 0.6930
Epoch 16/25: Validation Loss: 0.6987
Epoch 17/25: Validation Loss: 0.6949
Epoch 18/25: Validation Loss: 0.6968
Epoch 19/25: Validation Loss: 0.6973
Epoch 20/25: Validation Loss: 0.6970
Epoch 21/25: Validation Loss: 0.6963
Epoch 22/25

[I 2024-10-03 19:43:28,552] Trial 1 finished with value: 0.6914852499961853 and parameters: {'learning_rate': 0.0005600848988312334, 'momentum': 0.6327792322084209, 'triplet_margin': 0.6189066125747088, 'classification_loss_weight': 0.5434323177802199}. Best is trial 1 with value: 0.6914852499961853.


...

Epoch 25/25: Validation Loss: 0.1736

Trial 28:
Suggested Hyperparameters: learning_rate=0.0029585645690902575, momentum=0.5769189560176743, triplet_margin=1.1490398651250644, classification_loss_weight=0.6464878726686589
Epoch 1/25: Validation Loss: 0.6937
Epoch 2/25: Validation Loss: 0.6892
Epoch 3/25: Validation Loss: 0.6950
Epoch 4/25: Validation Loss: 0.6953
Epoch 5/25: Validation Loss: 0.6902
Epoch 6/25: Validation Loss: 0.6867
Epoch 7/25: Validation Loss: 0.6901
Epoch 8/25: Validation Loss: 0.6969
Epoch 9/25: Validation Loss: 0.6909
Epoch 10/25: Validation Loss: 0.6947
Epoch 11/25: Validation Loss: 0.6932
Epoch 12/25: Validation Loss: 0.6916
Epoch 13/25: Validation Loss: 0.6917
Epoch 14/25: Validation Loss: 0.6970
Epoch 15/25: Validation Loss: 0.6882
Epoch 16/25: Validation Loss: 0.6865
Epoch 17/25: Validation Loss: 0.6853
Epoch 18/25: Validation Loss: 0.6868
Epoch 19/25: Validation Loss: 0.6812
Epoch 20/25: Validation Loss: 0.6729
Epoch 21/25: Validation Loss: 0.6639
Epoch 22/2

[I 2024-10-03 20:38:06,601] Trial 28 finished with value: 0.6495035886764526 and parameters: {'learning_rate': 0.0029585645690902575, 'momentum': 0.5769189560176743, 'triplet_margin': 1.1490398651250644, 'classification_loss_weight': 0.6464878726686589}. Best is trial 27 with value: 0.17358753234148025.


Epoch 25/25: Validation Loss: 0.6504

Trial 29:
Suggested Hyperparameters: learning_rate=0.02766420584125575, momentum=0.7414515378501716, triplet_margin=1.5156401875457117, classification_loss_weight=0.4709551861759709
Epoch 1/25: Validation Loss: 0.7007
Epoch 2/25: Validation Loss: 0.7069
Epoch 3/25: Validation Loss: 0.7107
Epoch 4/25: Validation Loss: 0.6919
Epoch 5/25: Validation Loss: 0.6928
Epoch 6/25: Validation Loss: 0.6966
Epoch 7/25: Validation Loss: 0.6898
Epoch 8/25: Validation Loss: 0.6904
Epoch 9/25: Validation Loss: 0.6887
Epoch 10/25: Validation Loss: 0.6924
Epoch 11/25: Validation Loss: 0.6880
Epoch 12/25: Validation Loss: 0.6781
Epoch 13/25: Validation Loss: 0.6720
Epoch 14/25: Validation Loss: 0.6767
Epoch 15/25: Validation Loss: 0.6610
Epoch 16/25: Validation Loss: 0.6370
Epoch 17/25: Validation Loss: 0.6445
Epoch 18/25: Validation Loss: 0.6583
Epoch 19/25: Validation Loss: 0.6758
Epoch 20/25: Validation Loss: 0.6601
Epoch 21/25: Validation Loss: 0.6464
Epoch 22/25:

[I 2024-10-03 20:40:06,275] Trial 29 finished with value: 0.6257834553718566 and parameters: {'learning_rate': 0.02766420584125575, 'momentum': 0.7414515378501716, 'triplet_margin': 1.5156401875457117, 'classification_loss_weight': 0.4709551861759709}. Best is trial 27 with value: 0.17358753234148025.


Epoch 25/25: Validation Loss: 0.6516
Best hyperparameters:  {'learning_rate': 0.014803213977973965, 'momentum': 0.5079861758547359, 'triplet_margin': 1.703068819656492, 'classification_loss_weight': 0.6232609307249704}
Best validation loss:  0.17358753234148025


Best hyperparameters:  {'learning_rate': 0.014803213977973965, 'momentum': 0.5079861758547359, 'triplet_margin': 1.703068819656492, 'classification_loss_weight': 0.6232609307249704}
Best validation loss:  0.17358753234148025